## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Для открытия файла используем функцию `read_csv()` библиотеки pandas

In [1]:
# Импортируем библиотеку
import pandas as pd

# Создадим URL-адрес
url = '/datasets/data.csv'

# Загрузим набор данных
df = pd.read_csv(url)

# Взглянем на первые пять строк
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
# Взглянем на последние пять строк
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


В этом фрейме данных можно заметить три важных момента:

* во-первых, во фрейме данных каждая строка соответствует одному наблюдению, а каждый столбец соответствует одному признаку;
* во-вторых, каждый столбец содержит имя (`children`,	`days_employed`,	`dob_years`	 и т.д.), а каждая строка содержит индексный номер (`0`, `1` и т.д.), которые можно использовать для выбора и управления наблюдениями и признаками;
* в-третьих, столбцы `education` и `education_id` содержат одну и туже информацию в разных форматах, также как и столбцы `family_status` и `family_status_id`); например, в столбце `education` уровень образования клиентов обозначается `высшее`, `среднее` и т.д., в то время как в столбце `education_id` уровень образования обозначается `0`, `1` и т.д. Чтобы признаки были уникальными, один из дублирующих значниея столбцов можно удалить.

Посмотрим общую информацию по набору данных. Для начала взглянем на количество строк и столбцов. Для этого используем атрибут датафрейма `shape`

In [3]:
df.shape

(21525, 12)

Видим, что данный датафрейм содержит `21525` строк (объектов/наблюдений) и `12` столбцов (признаков)

Для более полной информации используем метод `info()`

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В возращаемом результате метода `info()` можно заметить, что не у всех признаков (столбцов) количество значений совпадает с общим количеством строк. Например, количество значений в столбце `days_employed` равно `19351`, что говорит о наличии пропущенных значений (приблизительно около 10%). Аналогично и в столбце `total_income`. Необходимо более подробно исследовать данные в этих столбцах, чтобы принять решение, что делать с пропущенными значениями.

Подсчитаем количество пропущеных значений. Для этого используем метод `isna()` для поиска пропусков и метод `sum()` для их подсчета

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Можно увидеть, что количество пропущеных значений в столбцах `days_employed` и `total_income ` равно `2174`

Также предварительно посмотрим на наличие выбросов в числовых данных. Для этого используем метод `describe()`

In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В данном наборе данных можно заметить некоторые аномалии:
* во-первых, минимальное количество детей в столбце `children` (строка: `min`) равное `-1`. В реальности так не бывает, возможно здесь в данных закралась ошибка; также вывызает вопрос значение максимального количества детей (строка `max`) равное `20` – в этом случае, с учетом того, что значение в третьем квартиле равно `1` выглядит как выброс, хотя не исключено, что это возможно и вовсе не ошибка, такое вполне может быть, хотя и крайне редко. Поскольку по условию задачи ***\"количество детей\"*** является одним из ключевых признаков необходимо будет рассмотреть эти случаи более подробно;
* во-вторых, отрицательные значения минимума, и первых трех квартилей (строки `25%`, `50%` и `75%`) столбца `days_employed` может говорить о большом количестве отрицательных значений данного признака; а значения максимума равного `401755.400475` дней, что приблизительно равно `1100` годам является очевидным выбросом; на что указывает большой разброс в данных – стандартное отклонение равное `140827.311974` дней или `385` лет (столбец `std`), а соответственно среднее равное `63046.497661` дней или `172` года также принимает аномальное значения (столбец `mean`),
* в-третьих, минимальный возраст клиента в столбце `dob_years` составляет `0` лет, что выглядит как ошибка, притом что остальные статистики говорят о нормальном распределении  значений.

Значения столбца `total_income` на первый взгляд кажутся чистыми. Хотя максимальное значение заработной платы в месяц в размере `2 265 604` ₽ выглядит чрезмерно большим более чем в 10 раз больше значения в третьем квартиле, необходимо будет внимательно изучить этот признак. 

Категориальные данные (`education_id`, `family_status_id`,	`debt`) по имеющимся статистикам метода `describe()` рассматривать не будем.

Посмотрим сколько отрицательных значений содержит столбец `days_employed`. Для этого используем условное выражение:

In [7]:
df[df['days_employed'] < 0]['days_employed'].count()

15906

15906 значений в столбце `days_employed` имеют отрицательные значения – это приблизительно 82% всех значений.

Посмотрим отдельно на неотрицательные значения, для удобства переведем значения из дней в годы:

In [8]:
df[df['days_employed'] >= 0]['days_employed'].apply(lambda x: x / 365).describe()

count    3445.000000
mean     1000.011808
std        57.739771
min       900.626632
25%       949.697024
50%      1000.584401
75%      1049.990258
max      1100.699727
Name: days_employed, dtype: float64

Из возвращаемого результата метода `describe()` видно, что значения трудового стажа слишком завышены: среднее значение равно 1000 лет при стандартном отклонении всего 57.7; такие данные непригодны для анализа.

Поскольку минимальное неотрицательное значение в столбце `days_employed` равно 328728.72 дней (или 900 лет), можно сделать вывод, что в датафрейме отсутствуют строки, которые укладываются в реальные значения трудового стажа, для примера возьмем трудовой стаж от 0 до 47 лет. 

Убедимся в этом:

In [9]:
df[(df['days_employed'] >= 0) & (df['days_employed'] <= 47*365)]['days_employed'].count()

0

Действительно, таких строк нет. Т.е. все значения в столбце `days_employed` не пригодны для анализа. Для того, чтобы привести значения в столбце в соответствие необходимо разобраться, почему данные в данном столбце именно такие. Возможно причина в неправильном маштабе, например данные указаны не в днях, а в часах; возможно – это произошло в следствие ошибки импорта данных и т.д. Такие вопросы лучше адресовать к специалистам, которые эти данные собирали. В любом случае, исходя из условия задачи и той информации, которая содержится в условии, этими данными можно пренебречь.
Поэтому столбец `days_employed` предлагается удалить из датафрейма.

Для удаления столбца используем метод `drop()` с параметром `axis='columns'`:

In [10]:
df.drop(labels='days_employed', axis='columns', inplace=True)

Другими кандидатами на удаление являются полные дубликаты строк в датафрейме. Посмотрим, есть ли в наборе данных полные дубликаты, то есть такие строки, в которых значения в одном и том же столбце равны. Для этого используем метод `duplicated()` и подсчитаем количество всех дубликатом с помощью метода `sum()`

In [11]:
df.duplicated().sum()

54

Как мы видим в нашем наборе данных имеется 54 дубликата.

### Вывод

1. Мы имеем набор данных о клиентах банка размером 21525 строк (объектов/наблюдений) и 12 столбцов (признаков). Из них 54 строки являются дубликатами.
2. В наборе данных есть две пары дублирующих друг друга категориальных признаков: `education` – `education_id` и `family_status` – `family_status_id`, значения которых записаны в разных форматах.
3. Также в наборе данных есть 2174 пропуска в столбцах `days_employed` и `total_income` в каждом.
4. В наборе данных есть ошибки и выбросы. Например, в столбцу `days_employed` отрицательное количество детей и отрицательное значение трудового стажа, или невозможное значение трудового стажа (1100 лет). Данный столбец, как зашумленный и не представляющий пользы в анализе данных на текущий этап, был удален из датафрейма.

***

### Шаг 2. Предобработка данных

### Обработка пропусков

Как было показано выше пропущенные зачения в дата фрейме содержат столбцы `days_employed` и `total_income`. Столбец `days_employed` был удален на предыдущем шаге, посмотрим подробнее данные в столбце `total_income`. Для начала посмотрим общие статистики. Используем метод `describe()` для объекта типа `Series`:

In [12]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Видно, что ненулевых значений в столбце `total_income` содержится 19351. Посмотрим являются ли пропусками остальные значения, используем метод `isna()`:

In [13]:
df['total_income'].isna().sum()

2174

Или в относительных величинах:

In [14]:
percentage_of_missing_values = df['total_income'].isna().sum() / len(df['total_income'])
print(f'Доля пропущенных значений: {percentage_of_missing_values:.2%}')

Доля пропущенных значений: 10.10%


10% – достаточно большое количество строк с пропусками для того, чтобы их просто удалить. Выберем стратегию заполнения пропущенных значений. Для начала, посмотрим на строки с пропусками. Выясним, есть ли связь меджу пропущенными значениями и другими признаками. Сгруппируем датафрейм с пропущенными значения по уровню образования и сохраним его в отдельную переменную `df_with_nan`:

In [15]:
df_with_nan = df[df['total_income'].isna()]

Сгруппируем получивщийся дата фрейм по уровню образования и посмотрим на распределение по количеству строк для каждой группы:

In [16]:
df_with_nan.groupby(by='education_id')['education_id'].count().sort_values(ascending=False)

education_id
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64

Больше всего строк с пропущенными значениями приходится на группу с индексом `1`. Интересно является ли эта группа более многочисленной для всей выборки?

In [17]:
df.groupby(by='education_id')['education_id'].count().sort_values(ascending=False)

education_id
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Да, более того, и распределение по группам приблизительно совпадает и относится к строкам с пустыми значениями именно как 100 к 10. Следовательно связи между уровнем образования и фактом наличия пропущенных значений нет.

Аналогично посмотрим для признаков `family_status_id`,	`gender`,	`income_type` и	`debt`

In [18]:
# Автоматизируем процесс
# Сгруппируем имена столбцов, в кторых будет проходить проверка в список
target_columns = ['family_status_id',  'gender',  'income_type',  'debt']

# Создадим функцию для сравнения распределения по признакам во всем дата фрейме и датафрейме с пропусками
def verify(column_name: str):
    """
    Функция сравнивает распределение количества строк в группах по целевому признаку
    между всей выборкой из дата фрейма и дата фреймом с пропущенными значениями
    
    Распределение сравнивается как отношение количества во всей выборке 
    к количеству в выборке с пропущенными значениями.
    
    Нормативное значение даннорго отношения лежит в границах интервала [8, 12],
    так как отношение между количеством строк во всей выборке к количеству строк
    в выборке с пропусками равняется 10, а сам интервал расширим на 20%.
    
    Функция возвращает True, если результат вычислений поапдает в нормативное значение
    """
    
    # Запишем значения количества строк в целевой группе в список для каждой выборки
    counts_in_df = [count 
                    for count 
                    in df.groupby(by=column_name)[column_name].count().sort_values(ascending=False)]
    counts_in_df_with_nan = [count 
                             for count 
                             in df_with_nan.groupby(by=column_name)[column_name].count().sort_values(ascending=False)]
    
    # Запишем результат отношения количества строк во всей выборке к количеству строк в выборке с пропусками в список
    result = []
    for i in range(min(len(counts_in_df_with_nan), len(counts_in_df))):
        result.append(counts_in_df[i] / counts_in_df_with_nan[i])
    
    # Вычислим среднее арифметическое результатов и с равним его с нормативными значениями
    return 8 <= sum(result) / len(result) <= 12
    

Посмотрим на результат работы функции `verify()`:

In [19]:
for target_column in target_columns:
    verify_ = verify(target_column)
    print(f'Для столбца {target_column} распределение: {verify_}')

Для столбца family_status_id распределение: True
Для столбца gender распределение: True
Для столбца income_type распределение: True
Для столбца debt распределение: True


Видно, что для всех признаков распределение для датафрема с пропусками является приблизительно таким же, как и для всего датафрейма. Значит наличие пропусков не связано ни с одним из признаков в столбцах: `education_id`, `family_status_id`,  `gender`,  `income_type`,  `debt` и нормально распределено по всему набору данных. Поэтому для импутации пропущенных зачений можно воспользоваться заменой пропусков на среднее значение.

Следует отметить, что заполнение пропусков средним – довольно топорная стратегия, которая больше подходит для быстрого заполнения больших объемов данных. Не всегда импутированное значение близко у истинному, особенно когда данные распределены ненормально и имеются явные выбросы. В нашем случае данные распределены нормально, поэтому такой подход в рамках решения данной задачи вполне уместен. Его можно усовершенствовать, рассчитав средние значения по одному из категориальных признаков, например по `income_type`, а затем, строкам с пропусками, у которых будет определенный `income_type` присваивать средние по данной категории.

Альтернативным методом импутации пропущенных значений – является применение метода $k$ ближайших соседей для предсказания пропущенных значений. Сохраним значение столбца `total_income` с пропущенными значениями для дальнейшего сравнения двух методов импутации:

In [20]:
# Создать глубоку копию значений столбца total_income

import copy

features = copy.deepcopy(df['total_income'].values)
pd.Series(features).isna().sum()

2174

##### Метод импутации на основе среднего значения

Рассчитаем средние показатели по категориям `income_type`:


In [21]:
df.groupby(by='income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Как видим средние значения достаточно сильно различаются. Создадим функцию, которая будет импутировать средние значения по категориям в строки с пропущенными значениями, относящиеся к схожей категории:

In [22]:
def impute_mean(category_name):
    """
    Функция импутирует среднее значение в группе в строки с пропусками
    """
        
    # Вычислить среднее значение в группе
    mean_income = df[df['income_type'] == category_name]['total_income'].mean()
    
    # Присвоить среднее значения в строки с пропущенными значениями
    df.loc[(df['income_type'] == category_name)
          & (df['total_income'].isna()), 'total_income'] = mean_income

Пройдем циклом по всем категориям и заменим пропуски на среднее значения для каждой категории с помощью функции `impute_mean()`:

In [23]:
for income_type in df['income_type'].unique():
    impute_mean(income_type)

Посмотрим на результат выполнения функции. Посмотрим на количество пропусков в столбце `total_income`:

In [24]:
df['total_income'].isna().sum()

0

Их нет, сравним средние значения по столбцу `total_income` до импутации и после, до импутации среднее значение было равно `167422.3`:

In [25]:
df['total_income'].mean()

167395.91574102986

Как видим среднее значение несколько отклонилось от того, что было до импутации.

##### Метод $k$ ближайших соседей

Применим метод $k$ ближайших соседей для предсказания пропущенных значений. Для этого признаки с пропущенными значениями рассмтаривается как вектор целей и оставшееся подмножество признаков используется для предсказания пропущенных значений. Хотя для вычисления значений можно применять широкий спектр алгоритмов машинного обучения, популярным выбором является алгоритм $k$ билижайших соседей (KNN). В данном алгоритме для предсказания пропущенного значения используется $k$ ближайших наблюдений в соотвтествие с определенной метрикой расстояния. Это разумно в небольших наборах данных, но становится довольно проблематичным, если набор данных содержит миллионы наблюдений.

Применим метод $k$ ближайших соседей для столбца `total`, сохраним результат вычислений в отдельных переменных:

In [26]:
# Загрузить библиотеку
from sklearn.impute import SimpleImputer

# Создать заполнитель
mean_imputer = SimpleImputer(strategy='mean')

# Импутировать значения
features_mean_imputed = mean_imputer.fit_transform(features.reshape(1, -1))

# Количество пропусков в столбце до импутации
pd.Series(features).isna().sum()

2174

In [27]:
# Количество пропусков после импутации
pd.Series(features_mean_imputed[0]).isna().sum()

0

Значения импутированы, сравним среднее значение выборки до и после импутации:

In [28]:
pd.Series(features).mean()

167422.30220817294

In [29]:
pd.Series(features_mean_imputed[0]).mean()

167422.30220817297

В данном случае метод сработал гораздо точнее, среднее значение по выборке с пропусками приблизительно равно выборке с импутированными значениями. Можно сказать, что метод $k$ ближайших соседей точнее импутации на основании среднего значения, даже усовершенстованного нами до категорийных средних значений.

### Вывод

1. В столбце `total_income` было пропущено 2174 значения, что составило 10% от всех записей дата фрейма. 
2. Было проведено исследование на предмент зависимости пропусков от категорийных признаков `education_id`, `family_status_id`,  `gender`,  `income_type`, `debt`. Было установлено, что зависимости наличия и распределения пропусков от категорийных признаков нет. Данные в подвыборке с пропущенными значениями распределены по категориям также как и во всей выборке. 
3. Поскольку распределение данных по столбу `total_income` близко к нормальному, в данных отсутствуют шумы и выбросы, было принято решение заменить пропущенные значения на среднее значение по столбцу `total_income`.
4. Также было замечено, что средние значения `total_income` по категориям `income_type` сильно разнятся – до 10 раз (от 53829.13 до 499163.15), поэтому для более точного импутирования было принято решение расчитать средние значения не по всей выборке, а по категориям `income_type` и заменить ими в соответствующих категориях пропущенные значения.
5. Дополнительно был протестирован альтернативный варинат импутации пропущенных значений на небольших выборках – метод $k$ ближайших соседей, который показал более точные результаты подстановки.

### Замена типа данных

Посмотрим еще раз на типы данных в датафрейме. Используем для этого атрибут `dtypes`

In [30]:
df['total_income'].dtypes

dtype('float64')

По условию задачи необходимо заменить вещественный тип данных на целочисленный. Как видно в дата фрейме только столбец `total_income` имеет тип значений `float64`. 

Заменим тип значений в столбце `total_income` на тип `int`. Для этого используем метод `astype()`:

In [31]:
df = df.astype({'total_income': 'int'}, errors='ignore')

В данном преобразовании типов мы использовали парметр `errors=` со значением `ignore`. Это означает, что для всех непредвиденных ситуаций, где по определенным причинам приведение типов окажется не возможным, метод `astype()` не прекратит свою работу из-за ошибок и исключений, а просто проигнорирует их. 

Альтернативная реализация такого поведения – использование конструкции `try` – `except`:

    try:
        df = df.astype({'total_income': 'int'})
    except:
        pass
    

Проверим результат изменения типа:

In [32]:
df['total_income'].dtypes

dtype('int64')

### Вывод

Преобразование типов бывает необходимо для оптимизации вычислений и использования памяти. В данном случае был преобразован тип значений в столбце `total_income` с `float64` к `int64`. Важно помнить, что при приобразовании типов могут возникать непредвиденные ситуации, когда такое преобразование невозможно выполнить. Для таких случаев необходимо использовать конструкцию `try`/`except` или параметр метод `astype()` `errors=` со значением `ignore`.

Также в библиотеки pandas есть еще несколько методов для преобразования типов:
* метод `to_numeric()` - предоставляет функциональные возможности для безопасного преобразования нечисловых типов (например, строк) в подходящий числовой тип, или подобне метод для преобразования строк в дату `to_datetime()` и `to_timedelta()`;
* метод `infer_objects()` - служебный метод для преобразования столбцов объектов, содержащих объекты Python, в тип pandas, если это возможно.

Эти альтернативные варианты для решения данной задачи не подходят.

### Обработка дубликатов

##### Удаление повторяющихся строк
Используем для этого метод `drop_duplucates()` и переиндексируем датафрейм с помощью метода `reset_index()`:

In [33]:
df.shape

(21525, 11)

In [34]:
# Сохраним количество сток в датафрейме до удаления дубликатов для проверки
rows_count = df.shape[0]

# Удалим дубликаты
df = df.drop_duplicates().reset_index()

In [35]:
rows_count - df.shape[0]

54

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
index               21471 non-null int64
children            21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


Как видим в датафрейме стало меньше на 54 строки

### Вывод

Важной частью анализа данных является выявление дублирующих значений и их удаление. Было выявлено 54 полных дубликата строк, а также две пары столбцов, дублировавших один признак разными типами значений. Строки было удалены из датафрейма, а столбцы оставлены.

Удаление дублирующихся строк необходимо для увеличения качества данных в датафрейме. По сути только уникальные строки представляют интерес с т.з. количества уникальной информации. Также наличие дубликатом может значительно влиять на основные статистики выборки данных, что в свою очередь может привести к ошибочным выводам.

Дублирование информации об одном признаке разными типами данных, например, строками и числами не искажает общие статистики, но в значительной степени увеличивает размер самого датафрейма. Удаление повториящихся столбцов для одного и того же признака является оптимизационной задачей. Например, строковые данные (в pandas тип `Object`) не имеют фиксированного размера выделяемой памяти, а значит занимают много места в памяти по сравнению с числовыми данными, например типа `int32`. Другая причина оставлять численные значения категориальных данных заключается в подготовки датафрейма для задач машинного обучения, в большинстве которых представление о категориальных данных является именно числовым.

### Лемматизация

Для лемматизации используем библиотеку pymystem3. Для начала лемматизируем все слова в слобце `purpose`. Используем метод `lemmatize()`

In [37]:
# Импортировать библиотеку
from pymystem3 import Mystem

# Создать лемматизатор
m = Mystem()

# Создать леммы
lemmas = df['purpose'].apply(m.lemmatize).values

Сохраним лемматизированные значения в новом столбце датафрейма `lemmas` для дальнейшей категоризации по целям выдачи кредита:

In [38]:
df['lemmas'] = df['purpose'].apply(m.lemmatize)

Соберем все лемматизированные слова вместе для очистки от знаков препинания и стоп слов. Поскольку знаков препинания в наших лемматизированных словах нет, удалим только символи пробела и перевода каретки. Символы пробела и символ перевода каретки удалим вручную:

In [39]:
purposes = []
for lemma in lemmas:
    for string in lemma:
        if string != ' ':
            if string != '\n':
                purposes.append(string)
        

In [40]:
len(purposes)

55067

Всего получилось 55067 слова. Многие из них повторяются, также среди них встречаются частые обще употребимые слова, так называемые стоп слова. Удалим стоп слова из целей. Используем для этого комплект естественно-языковых инструментов NLTK (Natural Language Toolkit for Python)

In [41]:
# Загрузить библиотеки
import nltk
from nltk.corpus import stopwords

# Сформировать набор стоп-слов
nltk.download('stopwords')
stop_words = stopwords.words('russian')

# Добавим часто встречающееся слово "свой", которого нет с списке стоп слов библиотеки NLTK
stop_words.append('свой')

# Удалить стоп-слова
clear_lemmas = [purpose for purpose in purposes if purpose not in stop_words]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
len(clear_lemmas)

45773

Список стал почти на 10000 слов короче. Примеры стоп слов:

In [43]:
stop_words[-10:]

['том',
 'нельзя',
 'такой',
 'им',
 'более',
 'всегда',
 'конечно',
 'всю',
 'между',
 'свой']

Создадим мешок слов по всем запросам для того, чтобы в дальнейшем категоризировать запросы по приоритетам. Для этого сохраним лемматизированные слова из столбца `purpose` в мультимножество `lemmas`:

In [44]:
# Загрузить библиотеку
from collections import Counter 

# Создать мешок слов
lemmas = Counter(clear_lemmas)

# Посмотреть самые популярные слова
lemmas.most_common(10)

[('недвижимость', 6353),
 ('покупка', 5900),
 ('жилье', 4461),
 ('автомобиль', 4308),
 ('образование', 4014),
 ('операция', 2604),
 ('свадьба', 2335),
 ('строительство', 1879),
 ('высокий', 1374),
 ('получение', 1315)]

Посмотрим на количество уникальных слов в мешке слов:

In [45]:
len(list(lemmas))

25

Всего уникальных слов оказалось 25 – это упрощает задачу категоризации по целям выдачи кредита. 

### Вывод

1. Лемматизация собирает разные флективные формы слова в группу, чтобы они могли анализироваться как одинаковые. Такой подход позволяет выделить суть в тексте, используя минимум слов. В нашем случае из общего количества слов в столбце `purpose` равным 55067 было выделено 25 уникальных слов, на основе которых можно категоризировать заемщиков по цели кредита.
2. Для достижения такого результата были использованы библиотеки: `pymystem3` и `nltk`. С помощью `pymystem3` были выделены леммы слов, а `nltk` помогла избавиться от стоп-слов. Символы перевода каретки и пробелы были удалены вручную.
3. Результат лемматизации был записан в специальную структуру данных python мультимножество `Counter`, которое позволяет хранить не только уникальные значения, но и количество их упоминаний в тексте.
4. А для каждой строки лемматизированные слова из столбца `purpose` были записаны в отдельный столбец `lemmas` для дальнейшей категоризации по целям кредита.

### Категоризация данных

В анализе данных ***категоризацией*** называют задачу отнесения наблюдений к одной из групп, называемых категориями. К основным особенностям данной задачи можно отнести:

* категория определяет лишь некоторые общие свойства объектов;
* задача категоризации менее определенна, чем, например, задача классификации;
* границы категорий, в отличие от классов, являются нечеткими, а сама категория определяется не формально, а сравнением с другими категориями.



Рассчмотрим несколько вариантов категоризации данных:

***по целям кредита:***

* автокредит;
* ипотека;
* потребительский кредит;
* кредит на образование;

***по количеству детей:***

* бездетных (`children == 0`);
* с детьми (`1 <= children < 3`);
* многодетных (`children >= 3`);

***по возрасту:***

* до 25 лет;
* от 26 до 55 лет;
* старше 56 лет;

***по уровню дохода:***

* доход до 70 000 ₽;
* доход от 70 000 ₽ до 120 000 ₽;
* доход от 120 000 ₽ до 200 000 ₽;
* доход от 200 000 ₽ до 600 000 ₽;
* доход свыше 600 000 ₽.

Такая категоризация поможет в дальнейшем выстроить простой алгоритм принятия решения о выдаче кредита, построенный на дереве решений.

##### Категоризация по целям кредита
Прежде, чем присупить к категоризации посмотрим повнимательнее на все уникальные леммы для столбца `purpose`:

In [46]:
unique_lemmas = list(lemmas)
unique_lemmas

['покупка',
 'жилье',
 'приобретение',
 'автомобиль',
 'дополнительный',
 'образование',
 'сыграть',
 'свадьба',
 'операция',
 'проведение',
 'семья',
 'недвижимость',
 'коммерческий',
 'жилой',
 'строительство',
 'собственный',
 'подержать',
 'заниматься',
 'сделка',
 'подержанный',
 'получение',
 'высокий',
 'профильный',
 'сдача',
 'ремонт']

Можно заметить, что среди списка уникальных слов встречаются прилагательные, глаголы и существительные. Для категоризации значений столбца `purpose` по целям кредита достаточно будет оставить в списке только существительные, которые являются достаточными для определения категорий. А такие слова как `'дополнительный'`, `'покупка'`, `'приобретение'`, `'собственный'` и пр. можно исключить. Единственное исключения для целей категоризации, могут составить прилагательные `'коммерческий'` и `'жилой'` для целей приобретения недвижимости, т.к. свидетельстуют о принадлежности цели кредита к разным категориям. В данной задаче категоризировать подробно ипотеку на две подкатегории мы не будем, и сведем все запросы по ипотеке в одну категорию.

Удалим все слова не являющиеся существительными из списка `unique_lemmas`. Для используем для этого метод `analyze()` библиотки pymystem3.

In [47]:
' '.join(lemmas)

'покупка жилье приобретение автомобиль дополнительный образование сыграть свадьба операция проведение семья недвижимость коммерческий жилой строительство собственный подержать заниматься сделка подержанный получение высокий профильный сдача ремонт'

In [48]:
tagged_lemmas = m.analyze(' '.join(lemmas))

# Показать первые три части речи
tagged_lemmas[:3]

[{'analysis': [{'lex': 'покупка', 'wt': 1, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'покупка'},
 {'text': ' '},
 {'analysis': [{'lex': 'жилье', 'wt': 1, 'gr': 'S,ед,сред,неод=(пр|вин|им)'}],
  'text': 'жилье'}]

Создадим функцию, которая оставляет в списке уникальных лемм только существительные:

In [49]:
def save_nouns(list_):
    """
    Функция возвращает новый список, который содержит только имена существительные
    """
    result_list = []
    
    for i in range(0, len(list_), 2):
        if list_[i]['analysis'][0]['gr'][0] == 'S':
            result_list.append(list_[i]['text'])
            
    return result_list

In [50]:
noun_lemmas = save_nouns(tagged_lemmas)
noun_lemmas

['покупка',
 'жилье',
 'приобретение',
 'автомобиль',
 'образование',
 'свадьба',
 'операция',
 'проведение',
 'семья',
 'недвижимость',
 'строительство',
 'сделка',
 'получение',
 'сдача',
 'ремонт']

Как видим, в списке остались только существительные. При этом среди них досих пор есть существительные, которые не несут сысмла для задачи категоризации по типу выдаваемого кредита, а именно: `'покупка'`, `'приобретение'`, `'проведение'`,  `'сделка'`, `'получение'`, `'сдача'`. Удалим их вручную:

In [51]:
del_strings = ['покупка', 'приобретение', 'проведение',  'сделка', 'получение', 'сдача']
noun_lemmas[:] = [string for string in noun_lemmas if string not in del_strings]
noun_lemmas

['жилье',
 'автомобиль',
 'образование',
 'свадьба',
 'операция',
 'семья',
 'недвижимость',
 'строительство',
 'ремонт']

Предварительная работа по выделению смысла в леммах сделана, теперь создадим фукцию, которая будет возвращать одну их категорий на основании лемматизированных целей:

In [52]:
def caterorize_purpose(list_):
    """
    Функция категоризирует кредиты по целям и возвращает имя категории кредита
    в зависимости от ключевых слов в запросе.
    
    Выделим 4 основные категории кредита:
    1) ипотека;
    2) автокредит;
    3) кредит на образование;
    4) потребительский кредит.
    
    В категорию потребительский кредит будем относить всё то,
    что однозначно не попадет в первые три категории.
    
    """
    for string in list_:
        if 'жилье' in list_ or 'недвижимость' in list_ or 'строительство' in list_:
            return 'ипотека'
        elif 'автомобиль' in list_:
            return 'автокредит'
        elif 'образование' in list_:
            return 'кредит на образование'
        else:
            return 'потребительский кредит'

Категоризируем данные по целям кредита и создадим столбец с категорией тип кредита с именем `credit_type`:

In [53]:
df['credit_type'] = df['lemmas'].apply(caterorize_purpose)

После категоризации по типу кредита столбцы `purpose` и `lemmas` можно удалить из датафрейма, так как они больше не несут полезной информации:

In [54]:
df.drop(['purpose', 'lemmas'], axis='columns', inplace=True)

Посмотрим по каким кредитам был больший процент задолженностей:

In [55]:
df.groupby(by='credit_type')['debt'].sum().sort_values(ascending=False)

credit_type
ипотека                   782
автокредит                403
кредит на образование     370
потребительский кредит    186
Name: debt, dtype: int64

Больше всего задолженностей было по ипотеке. Посмотрим на это в разрезе общего количества кредитов, выданных по данной категории:

In [56]:
df.groupby(by='credit_type')['credit_type'].count().sort_values(ascending=False)

credit_type
ипотека                   10814
автокредит                 4308
кредит на образование      4014
потребительский кредит     2335
Name: credit_type, dtype: int64

In [57]:
debt_by_credit_type = pd.pivot_table(data=df, columns='credit_type', values='debt')
debt_by_credit_type

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.093547,0.072314,0.092177,0.079657


Теперь картина изменилась, в относительных значениях больше всего долгов по автокредитам и кредитам на образование, а ипотека оказалась на последнем месте по задолженностям.

Тот же результат можно получить разделив один объект `Series` на другой, недостаток такого подхода проявится тогда, когда порядок столбцов будет отличаться. В нашем случае порядок в обоих случаях идентичный.

In [58]:
debts = df.groupby(by='credit_type')['debt'].sum().sort_values(ascending=False)
total = df.groupby(by='credit_type')['credit_type'].count().sort_values(ascending=False)
debts/total

credit_type
ипотека                   0.072314
автокредит                0.093547
кредит на образование     0.092177
потребительский кредит    0.079657
dtype: float64

##### Категоризация по количеству детей

Посмотрим еще раз на значения в столбце `children`:

In [59]:
df['children'].describe()

count    21471.000000
mean         0.539565
std          1.382978
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

Среди значений присутстует шум:
* ошибочное значение равное `-1`;
* выброс равный `20`.

Посмотрим много ли таких зашумленных значений. Для начала посмотрим сколько значений в столбце `children` меньше 0:

In [60]:
df[df['children'] < 0]['children'].count()

47

Оказывается, что целых 47 записей имеет отрицательные значения. Теперь посмотрим, сколько значений привышает условный норматив. Примем его равным 5. Семьи с пятью детьми всё же встречаются, хотя и являются редкостью.

In [61]:
df[df['children'] > 5]['children'].count()

76

Таких записей оказалось 76. Посмотрим на их основные статистики:

In [62]:
df[df['children'] > 5]['children'].describe()

count    76.0
mean     20.0
std       0.0
min      20.0
25%      20.0
50%      20.0
75%      20.0
max      20.0
Name: children, dtype: float64

Посмотрим какие вообще значения присутствуют в столбце `children` и их долю:

In [63]:
df['children'].value_counts(normalize=True)

 0     0.657026
 1     0.223977
 2     0.095571
 3     0.015370
 20    0.003540
-1     0.002189
 4     0.001910
 5     0.000419
Name: children, dtype: float64

Как видно это возращаемого результат функции `describe()` в выборке нет записей о количестве детей 6, 7 и др., кроме 20. Это может означать, что эти данные не являются выбросами, а скорее являются ошибками ввода. 

Можно сделать предположение, что значения `-1` и `20` это ошибки ввода или чтения файла с данными, где вместо `-1` должно быть `1`, а вместо `20` – `2`. И поскольку количество этих данных мало и составляет для `-1`: 2.2%, а для `20`:3,5% – их можно удалить из датафрейма, а не заменять значения, как было предположено выше, чтобы избежать ошибок.

Для подтверждения посмотрим какие кредиты брали клиенты из этого зашумленного подмножества данных и сравним его с общей картиной:

In [64]:
pd.pivot_table(data=df[df['children'] == -1], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.090909,0.0,0.0,0.0


In [65]:
pd.pivot_table(data=df[df['children'] == 20], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.125,0.055556,0.066667,0.333333


Как видно распределение данных по возратам кредита для групы клиентов, у которых `-1` или `20` детей отличется от общего, сравним эти значения с категориями клиентов с одним и двумя детьми:

In [66]:
pd.pivot_table(data=df[df['children'] == 1], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.106957,0.081733,0.103926,0.095685


In [67]:
pd.pivot_table(data=df[df['children'] == 2], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.120301,0.085024,0.114144,0.055814


Видим, что распределение для `-1` и `1` некоррелируется, распределение выглядит совершенно по-разному, а для `20` и `2` – наоборот выдглядит похоже. Поэтому значения в столбце `children` равные `-1`  – удалим, а значения равные `20` – заменим на `2`.

Удаляем строки со значением в столбце `children` равным `1`:

In [68]:
df = df[df['children'] >= 0]

Заменяем значения в строках созначение в столбце `children` равным `20` на `2`:

In [69]:
df['children'].replace(20, 2, inplace=True)

Проверим результаты наших действий:

In [70]:
df['children'].describe()

count    21424.000000
mean         0.479089
std          0.756328
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

Как видно теперь значения в столбце `children` очищены от ошибок.

Присупим к формированию категорийного признака по количеству детей. Разделим наш датасет на следующие группы:
* бездетные (`children == 0`);
* с детьми (`1 <= children < 3`);
* многодетные (`children >= 3`).

Создадим функцию, которая будет возвращать тип категории на основании количества детей:

In [71]:
def caterorize_children(children_count):
    """
    Функция категоризирует количество детей у клиентов по 3 основным категориям:
    1) бездетные, когда количество детей равно 0 -> возращает 0;
    2) с детьми, когда количество детей 1 или 2 -> возвращает 1;
    3) многодетный, когда количество детей больше 3 -> возвращает 2.    
    """
    if children_count == 0:
        return 0
    elif 1 <= children_count < 3:
        return 1
    else:
        return 2

Следует отметить, что такая категоризация подходит как для семейных клиентов, так и для клиентов, которые не имеют семьи.

Категоризируем данные по количеству и создадим столбец с категорией уровня многодетности с именем `children_type`:

In [72]:
df['children_type'] = df['children'].apply(caterorize_children)

##### Категоризация по опыту работы

Аналогично категоризируем клиентов банка по их возрасту. Посмотрим на значения в столбце `dob_years`:

In [73]:
df['dob_years'].describe()

count    21424.000000
mean        43.280620
std         12.577028
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Клиентов с орицательным возрастом нет, но есть с нулевым, посмотрим сколько их:

In [74]:
df[df['dob_years'] == 0]['dob_years'].count()

101

У 101 клиента возраст равен нулю. Это может быть ошибкой ввода или записи данных. Посмотрим на распределение этих клиентов по количеству долгов:

Сколько всего уникальных значений возраста есть в датафрейме:

In [75]:
len(df['dob_years'].unique())

58

Посмотрим на них:

In [76]:
df['dob_years'].value_counts(normalize=True, ascending=False)

35    0.028706
40    0.028239
41    0.028193
34    0.027913
38    0.027773
42    0.027726
33    0.027026
39    0.026699
31    0.025999
36    0.025859
29    0.025392
44    0.025345
30    0.025065
48    0.025019
37    0.024925
43    0.023852
50    0.023852
49    0.023712
32    0.023665
28    0.023385
45    0.023198
27    0.022965
56    0.022591
52    0.022591
47    0.022265
54    0.022125
46    0.021938
53    0.021378
58    0.021285
57    0.021191
51    0.020864
55    0.020631
59    0.020631
26    0.018997
60    0.017457
25    0.016664
61    0.016477
62    0.016290
63    0.012509
24    0.012323
64    0.012183
23    0.011763
65    0.009055
22    0.008542
66    0.008495
67    0.007795
21    0.005181
0     0.004714
68    0.004621
69    0.003921
70    0.003034
71    0.002707
20    0.002381
72    0.001540
19    0.000653
73    0.000373
74    0.000280
75    0.000047
Name: dob_years, dtype: float64

Посмотрим как распределены значения по наличи долгов у возрастной группы равной 0 лет:

In [77]:
pd.pivot_table(data=df[df['dob_years'] == 0], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.136364,0.06383,0.0,0.142857


Сравним с общим распределение по всем возрастам:

In [78]:
pd.pivot_table(data=df[df['dob_years'] >= 0], columns='credit_type', values='debt')

credit_type,автокредит,ипотека,кредит на образование,потребительский кредит
debt,0.093554,0.072475,0.092408,0.079726


Распределение не соответствует общему, из этого можно с делать вывод, что данные зануленны в случайном порядке. Посмотрим какое значение идет за нулевым:

In [79]:
sorted(df['dob_years'].unique())[1]

19

За нулевым значение идет значение `19`. Можно предположить, что возраст клиентов находится в интервале от 18 до 75 лет. Для замены нулевых значений предлагается их рассматривать как нулевые типа `np.nan` и полученные пропуски заполнить средними значениями:

In [80]:
# Сохранить среднее значение возраста в переменную mean_sob_years
mean_dob_years = df['dob_years'].mean()

# Заменим нулевые значения на средние
df['dob_years'].replace(0, mean_dob_years, inplace=True)

In [81]:
df['dob_years'].describe()

count    21424.000000
mean        43.484659
std         12.219191
min         19.000000
25%         33.750000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Категоризируем клиентов по возрасту следующим образом:
* до 25 лет;
* от 26 до 55 лет;
* старше 56 лет.

Как и прежде создадим функцию-категоризатор и добавим категории в новый столбец датафрейма:

In [82]:
def caterorize_dob_years(dob_years):
    """
    Функция категоризирует клиентов по возрасту на 3 основные категории:
    1) до 25 лет -> возращает 0;
    2) от 26 до 55 лет -> возвращает 1;
    3) свыше 55 лет -> возвращает 2.    
    """
    if dob_years < 25:
        return 0
    elif 26 <= dob_years < 55:
        return 1
    else:
        return 2

Добавляем новый столбец `dob_years_type`:

In [83]:
df['dob_years_type'] = df['dob_years'].apply(caterorize_dob_years)

##### Категоризация по уровню дохода

Аналогично, как и в предыдущих категоризациях. Сперва посмотрим на сами данные:

In [84]:
df['total_income'].describe()

count    2.142400e+04
mean     1.674495e+05
std      9.807975e+04
min      2.066700e+04
25%      1.076335e+05
50%      1.519020e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

На первый взгляд аномалий, шумов и выбросов в данных нет. Перейдем к созданию функции-категоризатора:

In [85]:
def caterorize_total_income(total_income):
    """
    Функция категоризирует клиентов по доходу на 5 основных категорий:
    1) до 70000 руб. -> возращает 0;
    2) от 70000 руб. до 120000 -> возвращает 1;
    3) от 120000 до 200000 руб. -> возвращает 2;
    4) от 200000 до 600000 руб. -> возвращает 3;
    5) свыше 600000 руб. -> возвращает 4
    
    """
    if total_income < 70000:
        return 0
    elif 70000 <= total_income < 120000:
        return 1
    elif 120000 <= total_income < 200000:
        return 2
    elif 200000 <= total_income < 600000:
        return 3
    else:
        return 4

Такое разделение необходимо на предварительном этапе, чтобы отсечь клиентов с низким доходом.

Создадим новый категорийный признак и запишем его значения в столбец `total_income_type`:

In [86]:
df['total_income_type'] = df['total_income'].apply(caterorize_total_income)

Прежде чем присупить к дальнейшему анализу посмотрим на те данные, которыми до этого не занимались.

Посмотрим на значения в столбце `gender`:

In [87]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Оказывается кроме очевидных значений `F` и `M` еще есть значение `XNA`. Посмотрим сколько записей содержит это значение:

In [88]:
df[df['gender'] == 'XNA']['gender'].count()

1

Оказывается всего одна запись, посмотрим на нее по внимательнее:

In [89]:
df[df['gender'] == 'XNA']

,index,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,credit_type,children_type,dob_years_type,total_income_type
10690,10701,0,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,ипотека,0,0,3


Посмотрим, является ли эта запись уникальной для типа клиента `компаньон`:

In [90]:
df[df['income_type'] == 'компаньон']['income_type'].count()

5071

Оказывается нет, записей со значением `income_type == компаньон` почти четверть из всех. Удалим эту запись, ее наличие или отсутствие не повлияет значительно на результат анализа:

In [91]:
df = df[df['gender'] != 'XNA']

Для задач машинного обучения было бы правиль представить категориальные признаки в числовом виде. Например заменить значения `F` и `M` в столбце `gender` на `0` и `1` соответственно. Используем метод `replace()`:

In [92]:
# Создадим словарь, состоящий из пар старое значение – новое значение
gender_mapper = {
    'F': 0,
    'M': 1,
}

# Заменим значения в столбце gender
df['gender'] = df['gender'].replace(gender_mapper)

Далее можно было бы также заменить описательные значения в столбце `income_type` на числовые значения, удалить один из дублирующих столбцов одного и тоже признака, например: в столбцах `education` и `education_id` или в `family_status`	и `family_status_id` одни и теже признаки представлены в разных форматах. Но мы не станем этого делать для удобства работы с именно с текстовым представлением признаков в датафрейме.

Вместо этого посмотрим повнимательнее на значения признаков в столбцах `education` и `education_id`, а также `family_status` и `family_status_id`

In [93]:
df['education_id'].unique()

array([0, 1, 2, 3, 4])

Видим, что все значения различны и не дублируются. проверим, есть ли пропуски в данных:

In [94]:
df['education_id'].isna().sum()

0

Пропусков нет – значит, все значения в столбце `education` закодированы и имеют числовое представление в столбце `education_id`.

Посмотрим на значения в столбце `education`:

In [95]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим, что значения `'среднее'`, `'Среднее'`, `'СРЕДНЕЕ'` – дублируются по смыслу, хотя и отличаются написанием, так, что для интерпритатора они являются различными. Исправим это, приведем их всех к нижнему регистру:

In [96]:
# Приведем значения в столбце education к нижнему регистру
df['education'] = df['education'].apply(lambda s: s.lower())

# Посмотрим на уникальные значения
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь все значения столбца `education` уникальные. Проверим, совпадает ли количество уникальных значений столбца `edication` и `education_id`:

In [97]:
len(df['education_id'].unique()) == len(df['education'].unique())

True

Количество уникальных значений совпадает, данные очищены от шума.

Аналогично сделаем для другой пары дублирующихся категориальных признаков: `family_status` – `family_status_id`

Посмотрим на значения признаков `family_status` и `family_status_id`:

In [98]:
df['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [99]:
df['family_status_id'].isna().sum()

0

Как и в предыдущем случае видим, что все значения уникальны и не содержат пропусков.

Посмотрим на описательное представление признака в столбце `family_status`:

In [100]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Видим, что данные не дублируются.

*Заметим, что значение признака \"Не женат / не замужем\" начинается с заглавной буквы в отличие от остальных значений, но в нашем случае это не принципиально.*

In [101]:
len(df['family_status']) == len(df['family_status_id'])

True

Количество значений признака также совпадает.

### Вывод

1. Категоризация – важная часть работы по анализу данных. Категории позволяют лучше структурировать данные и генерировать новые категориальные признаки, исходя из бизнес-логики предмета исследования.
2. В данной задаче были созданы 4 новых категориальных признака `credit_type`,	`children_type`,	`dob_years_type`,	`total_income_type`. На основании данных признаков можно построить простой алгоритм принятия решения о выдаче кредита.
3. В процессе создания новых признаков, данные на основе которых они создавались, были дополнительно более глубоко исследованы. В реузльтате такого исследования были выявлены и устранены пропуски, аномалии и выбросы в данных.
4. Также были исследованы остальные данные, которые не были категоризированы. В целом датасет готов к анализу.

### Шаг 3. Ответьте на вопросы

***- Есть ли зависимость между наличием детей и возвратом кредита в срок?***

Для ответа на этот впрос посмотрим количество долгов по кредитам для клиентов с различным количеством детей:

In [102]:
df.groupby(by='children')['debt'].sum() 

children
0    1063
1     444
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

В абсолютных значениях больше всего долгов имеют клиенты без детей, и с возрастание количества детей – количество должников уменьшается. Посмотрим на количество долгов по кредитам в относительных величинах:

In [103]:
(df.groupby(by='children')['debt'].sum() / df.groupby(by='children')['debt'].count()).sort_values(ascending=False)

children
4    0.097561
2    0.094925
1    0.092327
3    0.081818
0    0.075358
5    0.000000
Name: debt, dtype: float64

В относительных величинах – картина другая: больше всего долгов по кредитам у семей или несемейных с 4-мя детьми, хотя это и приблизительно равно с теми у кого 1, 2 или 3 ребенка. Другая ситуация с многодетными – у них долгов по кредитам нет. Несколько меньше долгов у бездетных клиентов.

Посмотрим на эти данные по категориям многодетности:

In [104]:
(df.groupby(by='children_type')['debt'].sum() / df.groupby(by='children_type')['debt'].count()).sort_values(ascending=False)

children_type
1    0.093124
2    0.081579
0    0.075358
Name: debt, dtype: float64

Распределение выглядит по другому:
* семьи с 1 или 2 детьми больше всех имеют задолженности по кредитам;
* многодетные семьи на втором месте, сказалось высокое количество долгов у семей с 4 детьми, а "идеальная" кредитная история семей с 5 детьми не смогла повлиять на общий показатель в силу своей малочисленности;
* бездетные клиенты оказались самыми дисциплинированными и имеют наименьшее количество долгов.

Дополнительно посмотрим на коэффициент корреляции Спирмана для категориальных данных. Коэффициент ранговой корреляции Спирмена – это непараметрический метод, который используется с целью статистического изучения связи между явлениями. В этом случае определяется фактическая степень параллелизма между двумя количественными рядами изучаемых признаков и дается оценка тесноты установленной связи с помощью количественно выраженного коэффициента.

Коэффициент корреляции Спирмена обладает следующими свойствами:
* коэффициент корреляции может принимать значения от минус единицы до единицы, причем при $rs=1$ имеет место строго прямая связь, а при $rs= -1$ – строго обратная связь;
* если коэффициент корреляции отрицательный, то имеет место обратная связь, если положительный, то – прямая связь;
* если коэффициент корреляции равен нулю, то связь между величинами практически отсутствует;
* чем ближе модуль коэффициента корреляции к единице, тем более сильной является связь между измеряемыми величинами.

In [105]:
df[['children_type', 'debt']].corr(method='spearman')

,children_type,debt
children_type,1.000000,0.029162
debt,0.029162,1.000000


### Вывод

Зависимость между количесвом детей и количеством долгов по кредиту слабая и прослеживается следующим образом: бездетные клиенты имеют меньше всего задолженностей, семьи с 1-2 детьми – самые большие, многодетные семьи на втором месте по количеству долгов. Особо выделяются две категории – семьи с 4 и семьи с 5 детьми – первые чемпионы по количеству долгов по кредитам, вторые наоборот – авбсолютно не имеют долгов!

***- Есть ли зависимость между семейным положением и возвратом кредита в срок?***

Аналогично посмотрим ка данные в контексте семейного положения:

In [106]:
df.groupby(by='family_status')['debt'].sum().sort_values(ascending=False)

family_status
женат / замужем          930
гражданский брак         388
Не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
Name: debt, dtype: int64

Предварительно семейные клиенты выглядят самыми недисциплинированными, в отличие от вдовцов или вдовиц. Посмотрим на относительные данные:

In [107]:
(df.groupby(by='family_status')['debt'].sum() / df.groupby(by='family_status')['debt'].count()).sort_values(ascending=False)

family_status
Не женат / не замужем    0.097683
гражданский брак         0.093337
женат / замужем          0.075518
в разводе                0.071369
вдовец / вдова           0.065969
Name: debt, dtype: float64

Относительные данные говорят о другом – самыми безответственными клиентами являются не женатые / не замужние, вдовцы / вдовицы – остаются в числе самых ответственных.

Посмотрим на коэффициент корреляции Спирмана:

In [108]:
df[['family_status_id', 'debt']].corr(method='spearman')

,family_status_id,debt
family_status_id,1.000000,0.023428
debt,0.023428,1.000000


### Вывод

Связь между семейным положением и задолженность по кредиту еще меньше, чем между количеством детей и кредитной задолженностью. Примерно одинаковое количество долгов имеют не женатые / не замужние и клиенты, находящиеся в гражданском браке. Также приблизительно равны значения кредитного долга у женатых и тех, кто находится в разводе. Чуть меньше задолжненость только у овдовевших клиентов.

***- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?***

Проанализируем аналогично данные в разрезе уровня дохода:

In [109]:
df.groupby(by='total_income_type')['debt'].sum().sort_values(ascending=False)

total_income_type
2    801
1    450
3    380
0    101
4      8
Name: debt, dtype: int64

Клиенты со средним доходом (от 120000 до 200000 руб.) больше всех имеют задолженностей по кредитам, в отличие от клиентов с самым маленьким и с самым большим доходом.

Посмотрим на относительные данные:

In [110]:
(df.groupby(by='total_income_type')['debt'].sum() / df.groupby(by='total_income_type')['debt'].count()).sort_values(ascending=False)

total_income_type
2    0.088694
1    0.083908
4    0.073394
3    0.069712
0    0.068754
Name: debt, dtype: float64

In [111]:
debts_income = (df.groupby(by='total_income')['debt'].sum() / df.groupby(by='total_income')['debt'].count()).sort_values(ascending=False)
debts_income.value_counts(normalize=True)

0.000000    0.915388
1.000000    0.078257
0.500000    0.005763
0.333333    0.000377
0.048276    0.000054
0.089059    0.000054
0.090994    0.000054
0.059761    0.000054
Name: debt, dtype: float64

В относительных цифрах клиенты со средним доходом также самые недисциплинированные (две категории от 70000 до 120000 и от 120000 до 200000 руб.), однако на третьем месте по количеству долгов переместились клиенты с самым высоким доходом (свыше 600000 руб.). Самыми же дисциплинированными отстались клиенты с самым низким уровнем доходов. 

Посмотрим на корреляцию Спирмана:

In [113]:
df[['total_income_type', 'debt']].corr(method='spearman')

,total_income_type,debt
total_income_type,1.000000,-0.010885
debt,-0.010885,1.000000


Отрицательное значение коэффициента корреляции говорит о наличии обратной связи между количеством долгов и уровнем дохода клиента, но поскольку значение корреляции близко к 0 говорить об устойчивой зависимости между этими признаками не приходится.

### Вывод

Между уровнем дохода и возвратом кредита в срок есть слабая обратная корреляция, но поскульку она близка к нулю – как таковой зависимости между этими признакми нет. Количество долгов по кредитам приблизительно равно между всеми группами по уровню доходов. Больше всего долгов у клиентов с средним доходом (до 200000 руб.), чуть меньше у клиентов с высоким уровнем доходов (более 200000 руб.), меньше всего у клиентов с доходом менее 70000 руб. Также видно, что большинство клиентов (92%) вовсе не имеют долгов, в то время как 7.8% имеют долги.

***- Как разные цели кредита влияют на его возврат в срок?***

Посмотрим на наличие долгов по кредиту в контексте целей кредита:

In [114]:
df.groupby(by='credit_type')['debt'].sum().sort_values(ascending=False)

credit_type
ипотека                   782
автокредит                402
кредит на образование     370
потребительский кредит    186
Name: debt, dtype: int64

Большинство долгов приходится на ипотечные кредиты, менбше всего на потребительские.

Посмотрим на относительные значения:

In [115]:
(df.groupby(by='credit_type')['debt'].sum() / df.groupby(by='credit_type')['debt'].count()).sort_values(ascending=False)

credit_type
автокредит                0.093554
кредит на образование     0.092408
потребительский кредит    0.079726
ипотека                   0.072481
Name: debt, dtype: float64

Видим как крединты по количеству долгов разделились на две группы: в лидерах по долгам – автокредит и кредит на образование, лучше дела обстоят с потребительским кредитом и ипотекой.

Рассчитаем корреляцию Спирмана, для этого добавим в датафрейм столбец с числовым представлением типа кредита.

In [116]:
# Создать словарь с парами значений старое строковое значение: новое числовое
credit_type_mapper = {}
for i, credit_type in enumerate(df['credit_type'].unique()):
    credit_type_mapper[credit_type] = i
credit_type_mapper

# Добавить новый столбец в датафрейм
df['credit_type_id'] = df['credit_type'].apply(lambda x: credit_type_mapper[x])

Теперь можем рассчитать коэффициент корреляции:

In [117]:
df[['credit_type_id', 'debt']].corr(method='spearman')

,credit_type_id,debt
credit_type_id,1.000000,0.026025
debt,0.026025,1.000000


Видно, что присутствует слабая прямая зависимость между признаками.

### Вывод

Между типом кредита и количеством задолженностей по нему есть слабая прямая корреляция. При этом также видно разделение на две группы: по автокредиту и кредиту на образование – большее количество должников, по потребительским кредитам и ипотеке – количество должников несколько меньше.

##### Поиск дополнительных зависимостей 

##### Поиск зависимости между образованием и наличием долгов по кредиту

In [120]:
(df.groupby(by='education')['debt'].sum() / df.groupby(by='education')['debt'].count()).sort_values(ascending=False)

education
начальное              0.109929
неоконченное высшее    0.091644
среднее                0.089961
высшее                 0.053033
ученая степень         0.000000
Name: debt, dtype: float64

Посчитаем коэффициент корреляции Спирмена:

In [121]:
df[['education_id', 'debt']].corr(method='spearman')

,education_id,debt
education_id,1.000000,0.056755
debt,0.056755,1.000000


Посмотрим на процет долгов по кредиту в разрезе уровня образования и доходов:

In [137]:
pd.pivot_table(data=df, columns='credit_type', values='debt', index=['education', 'total_income_type'])

credit_type                            автокредит   ипотека  \
education           total_income_type                         
высшее              0                    0.031250  0.086420   
                    1                    0.076087  0.033784   
                    2                    0.082725  0.056655   
                    3                    0.045570  0.042381   
                    4                    0.000000  0.043478   
начальное           0                    0.000000  0.000000   
                    1                    0.055556  0.137255   
                    2                    0.105263  0.145455   
                    3                    0.300000  0.055556   
неоконченное высшее 0                    0.000000  0.071429   
                    1                    0.090909  0.089888   
                    2                    0.095238  0.063694   
                    3                    0.115385  0.129310   
                    4                    1.000000  0.000000   
среднее             0                    0.098814  0.056361   
                    1                    0.096852  0.076186   
                    2                    0.108169  0.089385   
                    3                    0.100154  0.073082   
                    4                    0.000000  0.062500   
ученая степень      1                         NaN  0.000000   
                    2                         NaN  0.000000   
                    3                         NaN  0.000000   

credit_type                            кредит на образование  \
education           total_income_type                          
высшее              0                               0.043478   
                    1                               0.060403   
                    2                               0.064433   
                    3                               0.049587   
                    4                               0.200000   
начальное           0                               0.000000   
                    1                               0.117647   
                    2                               0.125000   
                    3                               0.000000   
неоконченное высшее 0                               0.000000   
                    1                               0.103448   
                    2                               0.093750   
                    3                               0.060606   
                    4                               1.000000   
среднее             0                               0.067460   
                    1                               0.116708   
                    2                               0.105777   
                    3                               0.094178   
                    4                               0.200000   
ученая степень      1                                    NaN   
                    2                                    NaN   
                    3                               0.000000   

credit_type                            потребительский кредит  
education           total_income_type                          
высшее              0                                0.000000  
                    1                                0.028302  
                    2                                0.059322  
                    3                                0.042553  
                    4                                0.000000  
начальное           0                                0.000000  
                    1                                0.200000  
                    2                                0.142857  
                    3                                0.166667  
неоконченное высшее 0                                0.000000  
                    1                                0.125000  
                    2                                0.062500  
                    3                                0

Посмотрим на процет долгов по кредиту в разрезе количества детей и доходов:

In [138]:
pd.pivot_table(data=df, columns='credit_type', values='debt', index=['children', 'total_income_type'])

credit_type                 автокредит   ипотека  кредит на образование  \
children total_income_type                                                
0        0                    0.073394  0.054924               0.068063   
         1                    0.089636  0.062783               0.103048   
         2                    0.088576  0.076715               0.088528   
         3                    0.078571  0.060045               0.068217   
         4                    0.142857  0.051282               0.272727   
1        0                    0.103448  0.057971               0.051724   
         1                    0.100000  0.086806               0.106061   
         2                    0.131514  0.084349               0.121447   
         3                    0.079545  0.078341               0.085973   
         4                    0.000000  0.066667               0.000000   
2        0                    0.222222  0.070175               0.057143   
         1                    0.089109  0.092527               0.150442   
         2                    0.137143  0.095890               0.089820   
         3                    0.110169  0.062718               0.118812   
         4                    0.000000  0.000000               0.500000   
3        0                    0.000000  0.166667               0.000000   
         1                    0.153846  0.022222               0.000000   
         2                    0.041667  0.119403               0.105263   
         3                    0.100000  0.046512               0.000000   
         4                         NaN  0.000000               0.000000   
4        1                    0.000000  0.250000               0.000000   
         2                    0.000000  0.133333               0.000000   
         3                    0.250000  0.000000                    NaN   
5        0                         NaN  0.000000                    NaN   
         1                    0.000000       NaN                    NaN   
         2                    0.000000  0.000000                    NaN   
         3                         NaN  0.000000               0.000000   

credit_type                 потребительский кредит  
children total_income_type                          
0        0                                0.058824  
         1                                0.071942  
         2                                0.083596  
         3                                0.067885  
         4                                0.000000  
1        0                                0.137931  
         1                                0.121429  
         2                                0.103139  
         3                                0.051471  
         4                                0.000000  
2        0                                0.066667  
         1                                0.103448  
         2                                0.066667  
         3                                0.033333  
         4                                0.000000  
3        0                                0.750000  
         1                                0.333333  
         2                                0.066667  
         3                                0.000000  
         4                                0.000000  
4        1                                0.000000  
         2                                0.000000  
         3                                0.000000  
5        0                                     NaN  
         1                                     NaN  
         2                                0.000000  
         3                                     NaN

Посмотрим на процет долгов по кредиту в разрезе количества детей и семейного положения:

In [139]:
pd.pivot_table(data=df, columns='credit_type', values='debt', index=['children', 'family_status'])

credit_type                     автокредит   ипотека  кредит на образование  \
children family_status                                                        
0        Не женат / не замужем    0.123552  0.073815               0.111597   
         в разводе                0.066667  0.067285               0.082278   
         вдовец / вдова           0.094527  0.046809               0.068182   
         гражданский брак         0.099291  0.078582               0.130112   
         женат / замужем          0.072078  0.065817               0.074542   
1        Не женат / не замужем    0.139785  0.113725               0.099010   
         в разводе                0.088235  0.071038               0.032787   
         вдовец / вдова           0.000000  0.081633               0.176471   
         гражданский брак         0.160377  0.122605               0.187500   
         женат / замужем          0.097810  0.071807               0.096447   
2        Не женат / не замужем    0.238095  0.083333               0.066667   
         в разводе                0.142857  0.080000               0.105263   
         вдовец / вдова           0.166667  0.125000               0.000000   
         гражданский брак         0.108108  0.129032               0.187500   
         женат / замужем          0.112760  0.080447               0.108571   
3        Не женат / не замужем    0.000000  0.500000               0.000000   
         в разводе                0.000000  0.100000                    NaN   
         вдовец / вдова                NaN  0.000000               0.000000   
         гражданский брак         0.400000  0.000000               0.166667   
         женат / замужем          0.060000  0.078014               0.051724   
4        Не женат / не замужем         NaN  1.000000               0.000000   
         в разводе                0.000000       NaN                    NaN   
         вдовец / вдова                NaN  0.000000                    NaN   
         гражданский брак         0.000000  0.000000                    NaN   
         женат / замужем          0.125000  0.133333               0.000000   
5        гражданский брак         0.000000       NaN                    NaN   
         женат / замужем          0.000000  0.000000               0.000000   

credit_type                     потребительский кредит  
children family_status                                  
0        Не женат / не замужем                     NaN  
         в разводе                                 NaN  
         вдовец / вдова                            NaN  
         гражданский брак                     0.074675  
         женат / замужем                           NaN  
1        Не женат / не замужем                     NaN  
         в разводе                                 NaN  
         вдовец / вдова                            NaN  
         гражданский брак                     0.095685  
         женат / замужем                           NaN  
2        Не женат / не замужем                     NaN  
         в разводе                                 NaN  
         вдовец / вдова                            NaN  
         гражданский брак                     0.066964  
         женат / замужем                           NaN  
3        Не женат / не замужем                     NaN  
         в разводе                                 NaN  
         вдовец / вдова                            NaN  
         гражданский брак                     0.156250  
         женат / замужем                           NaN  
4        Не женат / не замужем                     NaN  
         в разводе                                 NaN  
         вдовец / вдова                            NaN  
         гражданский брак                     0.000000  
         женат / замужем                           NaN  
5        гражданский брак                     0.000000  
         женат / замужем                           NaN

Посмотрим на процет долгов по кредиту в разрезе пола и типа доходов (женщины закодированы `0`, мужчины – `1`):

In [142]:
pd.pivot_table(data=df, columns='credit_type', values='debt', index=['gender', 'income_type'])

credit_type             автокредит   ипотека  кредит на образование  \
gender income_type                                                    
0      безработный             NaN  0.000000                    NaN   
       в декрете          1.000000       NaN                    NaN   
       госслужащий        0.072464  0.043165               0.081081   
       компаньон          0.069018  0.063012               0.061789   
       пенсионер          0.062121  0.043109               0.062185   
       предприниматель         NaN       NaN                    NaN   
       сотрудник          0.098951  0.074838               0.092417   
1      безработный             NaN  1.000000                    NaN   
       госслужащий        0.089744  0.060914               0.083333   
       компаньон          0.100503  0.070304               0.100592   
       пенсионер          0.074627  0.073034               0.088000   
       предприниматель         NaN  0.000000                    NaN   
       сотрудник          0.133253  0.104930               0.138614   
       студент                 NaN  0.000000                    NaN   

credit_type             потребительский кредит  
gender income_type                              
0      безработный                         NaN  
       в декрете                           NaN  
       госслужащий                    0.033058  
       компаньон                      0.084592  
       пенсионер                      0.060519  
       предприниматель                0.000000  
       сотрудник                      0.071429  
1      безработный                         NaN  
       госслужащий                    0.081081  
       компаньон                      0.122449  
       пенсионер                      0.049383  
       предприниматель                     NaN  
       сотрудник                      0.104677  
       студент                             NaN

Доля выданных кредитов

In [144]:
df['credit_type'].value_counts(normalize=True)

ипотека                   0.503618
автокредит                0.200579
кредит на образование     0.186902
потребительский кредит    0.108902
Name: credit_type, dtype: float64

Посмотрим на некоторые социально-демографические показатели (вторая колонка количество выданных кредитов):

*- уровень образования*

In [167]:
df.groupby(by='education')['credit_type'].count().sort_values(ascending=False)

education
среднее                15151
высшее                  5242
неоконченное высшее      742
начальное                282
ученая степень             6
Name: credit_type, dtype: int64

*- семейное положение*

In [168]:
df.groupby(by='family_status')['credit_type'].count().sort_values(ascending=False)

family_status
женат / замужем          12315
гражданский брак          4157
Не женат / не замужем     2805
в разводе                 1191
вдовец / вдова             955
Name: credit_type, dtype: int64

*- пол*

In [169]:
df.groupby(by='gender')['credit_type'].count().sort_values(ascending=False)

gender
0    14154
1     7269
Name: credit_type, dtype: int64

*- тип занятости*

In [170]:
df.groupby(by='income_type')['credit_type'].count().sort_values(ascending=False)

income_type
сотрудник          11065
компаньон           5070
пенсионер           3829
госслужащий         1453
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: credit_type, dtype: int64

*- уровень дохода*

In [171]:
df.groupby(by='total_income_type')['credit_type'].count().sort_values(ascending=False)

total_income_type
2    9031
3    5451
1    5363
0    1469
4     109
Name: credit_type, dtype: int64

*- стаж*

In [182]:
df.groupby(by='dob_years_type')['credit_type'].count().sort_values(ascending=False)

dob_years_type
1    15412
2     5137
0      874
Name: credit_type, dtype: int64

*- возраст*

In [186]:
sum(df.groupby(by='dob_years')['credit_type'].count().sort_values(ascending=False).index[:10]) / 10

36.9

*- количество детей*

In [187]:
df.groupby(by='children')['credit_type'].count().sort_values(ascending=False)

children
0    14106
1     4809
2     2128
3      330
4       41
5        9
Name: credit_type, dtype: int64

### Шаг 4. Общий вывод

Поиск зависимостей и закономерностей в данных – важная часть задачи кредитного скоринга. Для построения высокоточного предиктивного алгоритма, который сможет предсказывать вернет ли заемщик кредит в срок и без задержек или нет, необходимо выявить ключевые признаки, которые в большей мере влияют на целевую переменную. Из всех представленных признаков в датафрейме ***более всего коррелирует с признаком наличия задолженности по кредиту – признак, описывающий уровень образования у клиента***. Чем ниже уровень образования, тем больший процент задержек по платежам по кредиту есть у клиентов. При этом следует уточнить, что все признаки, включая и уровень образования в общем имеют слабую корреляционную связь с задолженностью по кредиту. Между уровнем дохода и задолженностью по кредиту корреляция и вовсе близка к нулю.

Можно выдвинуть гипотезу, что большей корреляцией могут обладать группы категориальных признаков, а не каждый из них по отдельности. *Некоторые пары признаков для примера представлены в таблицах выше.* Тем не менее уже сейчас можно построить простой алгоритм на основе решающих деревье для предсказания вероятности возврата кредита без задержек на данной группе признаков.

Общие характеристики выданных кредитов банка можно описать следующим образом:
* больше всего было выдано ипотечных кредитов (50.3%), на втором месте автокредит (20.1%), на третьем – кредит на образование (18.7%), последнее место у потребительского кредита (10.9%);
* в целом 91.5% клиентов со 100% вероятностью выплачивает кредиты в срок без задержек, 7.8% – со 100% задержками, 0.6% – с задержкой в 50%, остальные – 0.1% с задеркой с вероятностью 33% и менее;
* больше всего кредито было выдано клиентам со средним образованием, меньше всего клиентам с ученой степенью;
* женщинам было выдано в два раза больше кредитов, чем мужчинам;
* основыми клиентами являются сотрудники и компаньоны, а также пенсионеры и госслужащие, предпринимателям было вылано всего 2 кредита;
* бездетным выдается больше кредитов, чем клиентам с детьми.



<div align="center" border="1px"><strong><br/><br/>Портрет ключевого клиента выглядит следующим образом:<br/>   
женщина, 37 лет, сотрудник со средним образованием, замужем, без детей, с доходом от 120000₽ до 200000₽<br/><br/></strong></div>


Другие интересные особенности, которые были замечены в процессе исследования данных:
* чем больше уровень дохода у клиентов с начальным образованием, тем выше доля долгов по кредиту; этокасается всех типов кредитов, но не кредита на образование – клиенты с  самым высоким доходом всегда закрывают кредит на образование без долгов;
* клиенты с ученой степенью всегда погашают кредиты в срок;
* у клиентов со средним образованием и показатели по возврату кредитов в срок – тоже средние, это релевантно для всех типов кредитов;
* клиенты с высшим образованием с ростом доходов быстрее закрывают ипотеку, во всяком случае доля долгов по ипотекчному кредиту у этой группы с ростом доходов падает;
* у клиентов с низким доходом с ростом количества детей растет доля просрочек по ипотечным кредитам;
* клиенты с большим количеством детей (больше 3-х) не имеют задолженностей по образовательному и потребительскому кредитам;
* все клиенты, имеющие 3-х и более детей – семейные пары и имеют в основном задолженности по авткредиту или по ипотеке;
* безработные мужчины могут получить ипотеку (и иметь задолженность по ней в 100% случаев), а безработные женщины могут получить автокредит, также со 100% вероятностью задержать сроки выплат по кредиту;
* декрет как правило означает просрочку платежей, в основом за автомобиль;
* госслужащие мужчины имеют больше просрочек по кредту, чем их коллеги женщины;
* а вот пенсионерки наоборот, задерживают платежи в большей вероятностью, чем пенсионеры.

Итак, однозначного признака, от которого зависит возврат кредита не обнаружено. Наиболее коррелирует с вероятностью появления долгов по кредиту признак уровня образования, остальные признаки или имеют слубую корреляцию или не имеют ее вовсе.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.